In [1]:
# !pip install SQLAlchemy
# !pip install pyspark
# !pip install jupysql duckdb duckdb-engine
# !pip install delta-spark
# !pip install py4j
# !pip install -q findspark
# conda install -c conda-forge submitit 
# conda install corn, nbconvert
# !pip install graphviz
# !pip install hydra-core
# !pip install hydra-submitit-launcher
# !pip install submitit
# !pip install covalent-slurm-plugin
# !pip install ipynb-py-convert

In [2]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [3]:
import findspark
import pandas as pd
import numpy as np

import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

# import submitit

import time as t 
from datetime import date, datetime, timedelta
from joblib import Parallel, delayed
import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
# import covalent as ct

import subprocess
import re

findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# os.environ['PYDEVD_DISABLE_FILE_VALIDATION']=1

spark = SparkSession.builder \
       .master("local[2]") \
       .appName("MyApp") \
       .config("spark.driver.maxResultSize", "20g")\
       .config("spark.driver.memory", "100g")\
       .getOrCreate()
sc = spark.sparkContext

# sqlContext = SQLContext(sc)


spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
spark.sql("set spark.sql.execution.arrow.enabled=false")

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/28 09:50:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/28 09:50:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/28 09:50:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/10/28 09:50:40 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


DataFrame[key: string, value: string]

In [4]:
spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data.csv") \
          .createOrReplaceTempView("VINs_data")

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")

24/10/28 09:50:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
def whether_thisVIN_data_already_generated(thisVIN): 
    df_VINs_data = spark.sql("SELECT * FROM VINs_data")
    df_VINs_data = df_VINs_data.toDF("VIN", "TOTAL_ROWS")
    df_VINs_data_p = df_VINs_data.toPandas()
    for index, row in df_VINs_data_p.iterrows():
        if row['VIN'] == thisVIN:
          return 1
    return 0 

# Function to submit a SLURM job and return the job ID
def submit_job(script_path, thisVIN, account='vuh14_dibbs_sc', partition='sla-prio'):
    submit_command = f"sbatch --account={account} --partition={partition} {script_path} {thisVIN}"
    print("The submit command for the submitted job is: ")
    print(submit_command)
    result = subprocess.run(submit_command, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        job_id = re.findall(r'\d+', result.stdout)[0]
        print(f"Job {job_id} submitted successfully.")
        return job_id
    else:
        print("Job submission failed.")
        return None
 
# Function to check the status of multiple jobs
def check_jobs_status(job_ids):
    jobs_running = set(job_ids)
    while jobs_running:
        for job_id in list(jobs_running):
            status_command = f"squeue -j {job_id}"
            result = subprocess.run(status_command, shell=True, capture_output=True, text=True)
            if not result.stdout:  # If the job is no longer in the queue
                print(f"Job {job_id} completed.")
                jobs_running.remove(job_id)
        if jobs_running:
            print("Some jobs are still running...")
            print(jobs_running)
            t.sleep(1)  # Check every 5 seconds

# Function to retrieve job outputs
def get_jobs_output(output_files):
    for output_file in output_files:
        try:
            with open(output_file, "r") as file:
                output = file.read()
            print(f"Output from {output_file}:")
            print(output)
        except FileNotFoundError:
            print(f"Output file {output_file} not found.")

    

df_all_pop_VINs = spark.sql('select distinct VIN from population ') 
df_generated_pop_VINs = spark.sql('select * from VINs_data') 
df_generated_pop_VINs = df_generated_pop_VINs.toDF("VIN", "TOTAL_ROWS")


print("size of df_all_pop_VINs={0}, df_generated_pop_VINs = {1}".format(df_all_pop_VINs.count(), df_generated_pop_VINs.count()))    
    
if (df_all_pop_VINs.count() > df_generated_pop_VINs.count()):   

    population_num_of_rows = ' LIMIT 400 '

    df_pop_VINs = spark.sql('select distinct VIN from population order by RANDOM() {0} '.format(population_num_of_rows))  #where VIN like "'"1M1AN02Y2JM028028"'" ') #   % or VIN like "'"1M1AN02YXJM027581%"'" or VIN like "'"%"'" or VIN like "'"1M1AN02Y9JM027295%"'" {0} '.format(population_num_of_rows))   #  All rows in population table is: 322178
    df_pop_VINs_p = df_pop_VINs.toPandas()


    # Submitting multiple jobs
    job_ids = []
    output_files = []
    for index, row in df_pop_VINs_p.iterrows():
        thisVIN = row['VIN']
        if whether_thisVIN_data_already_generated(thisVIN)==1:
            print("This VIN = {}'s data is already generated.".format(thisVIN))
            continue
        else:
            print("This VIN = {}'s data is not generated.".format(thisVIN))
            aCurrentVIN = thisVIN 
            df_filtered_population_for_this_VIN = df_all_pop_VINs.filter(f.col('VIN')==aCurrentVIN)
            script_path = f"/storage/home/yqf5148/work/volvoPennState/Jobs/submit.sh"  # Assuming different scripts for each job
            output_file = f"/storage/home/yqf5148/work/volvoPennState/Jobs/log_test/result_{thisVIN}.txt"  # Assuming different output files for each job
            job_id = submit_job(script_path, thisVIN)

            if job_id:
                job_ids.append(job_id)
                output_files.append(output_file)

    # Check the status of all jobs 
    check_jobs_status(job_ids)

    # Retrieve and print outputs for all jobs
    get_jobs_output(output_files)

    # # check if there is any remaining VIN for which we have not generated data. In other words, if  df_all_pop_VINs > df_generated_pop_VINs we should continue generating dsata for the remaining VINs.
    # df_generated_pop_VINs = spark.sql('select * from VINs_data') 
    # df_generated_pop_VINs = df_generated_pop_VINs.toDF("VIN", "TOTAL_ROWS")


size of df_all_pop_VINs=322178, df_generated_pop_VINs = 481


This VIN = 4V4NC9EH0HN981742's data is not generated.
The submit command for the submitted job is: 
sbatch --account=vuh14_dibbs_sc --partition=sla-prio /storage/home/yqf5148/work/volvoPennState/Jobs/submit.sh 4V4NC9EH0HN981742
Job 25226938 submitted successfully.
This VIN = 1M2AX09C7FM021955's data is not generated.
The submit command for the submitted job is: 
sbatch --account=vuh14_dibbs_sc --partition=sla-prio /storage/home/yqf5148/work/volvoPennState/Jobs/submit.sh 1M2AX09C7FM021955
Job 25226939 submitted successfully.
This VIN = 1M1AN2GY9LM017678's data is not generated.
The submit command for the submitted job is: 
sbatch --account=vuh14_dibbs_sc --partition=sla-prio /storage/home/yqf5148/work/volvoPennState/Jobs/submit.sh 1M1AN2GY9LM017678
Job 25226940 submitted successfully.
This VIN = 4V4WC9EH3KN219842's data is not generated.
The submit command for the submitted job is: 
sbatch --account=vuh14_dibbs_sc --partition=sla-prio /storage/home/yqf5148/work/volvoPennState/Jobs/submi

In [ ]:
display(df_pop_VINs_p)